In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer

train_data = pd.read_csv("../dataSet/ml_case_training_data.csv", encoding='utf-8')
train_hist = pd.read_csv("../dataSet/ml_case_training_hist_data.csv", encoding='cp949')
train_out = pd.read_csv("../dataSet/ml_case_training_output.csv", encoding='utf-8')

In [3]:
clean = pd.merge(train_data, train_out, on="id")

In [4]:
#결측치 비율이 70퍼센트 이상이 칼럼 제거
clean = clean.drop([
    'campaign_disc_ele',
    'date_first_activ',
    'forecast_base_bill_ele',
    'forecast_base_bill_year',
    'forecast_bill_12m',
    'forecast_cons'
], axis=1)
clean.shape

(16096, 27)

In [5]:
#고객 회사 활동, 고객이 가입한 캠페인 칼럼의 결측치 새 카테고리로 분류
cols = ['activity_new', 'channel_sales']
clean[cols] = clean[cols].fillna('missing')


In [6]:
#카테고리를 워드임베딩하여 카테고리간의 거리를 고려할 수 있도록 한다. (관광업을 하는 고객과 머신러닝을 하는 회사 간의 차이가 머신러닝, 딥러닝을 하는 회사간의 거리보다 크다. 하지만 원 핫 인코딩은 둘 사이의 거리가 같다.)
#아쉽게 회사 활동이 코드로 치환되어있어서 패스
clean[['activity_new', 'channel_sales']].head(1)

,activity_new,channel_sales
0,esoiiifxdlbkcsluxmfuacbdckommixw,lmkebamcaaclubfxadlmueccxoimlema


In [7]:
#날짜 변환
import pandas as pd

datetype_col = ["date_activ", "date_end", "date_modif_prod", "date_renewal"]
base_date = pd.to_datetime('2000-01-01')

for col in datetype_col:
    clean[col] = pd.to_datetime(clean[col], errors='coerce')  # 문자열 → datetime, 결측은 NaT로 유지
    clean[col] = (clean[col] - base_date).dt.days             # 경과일수, 결측은 NaN
    clean[col] = clean[col].astype('Int64')                   # 결측치는 <NA>로, 나머지는 int형

In [8]:
#date_end 모두 이탈 안한 케이스. 
#계약시작일 + 이탈 안한 데이터들의 평균 계약기간으로 설정
이탈 = clean[clean['churn'] == 1]
유지 = clean[clean['churn'] == 0]

m = int((유지["date_end"] - 유지["date_activ"]).mean())
mask = clean["date_end"].isna()
clean.loc[mask, "date_end"] = clean.loc[mask, "date_activ"] + m

In [9]:
#날짜 칼럼의 결측치를 채워야하는데 평균 최빈값으로 대충 채우면 데이터의 무결성이 깨지게된다.
#계약 시작일 < 계약 종료일

In [10]:
# mask = clean["date_end"] < clean["date_renewal"]
# clean[mask].count()

In [11]:
clean[["date_activ", "date_end", "date_modif_prod", "date_renewal"]].head(10)

,date_activ,date_end,date_modif_prod,date_renewal
0,4694,6154,4694,5791
1,4914,6010,<NA>,5652
2,3520,6086,3520,5721
3,3758,5950,3758,5585
4,3741,5933,3741,5568
5,3750,5942,3750,5580
6,3665,5910,3665,5546
7,4360,6187,5783,5822
8,4353,6180,4353,5815
9,3763,5955,3763,5590


In [12]:
# 데이터 조건
#date_activ, date_renewal <= date_end

#특이점이 date_activ와 date_modif_prod가 같은 애들이 보인다.
#이탈자와 비이탈자의 date_activ와 date_modif_prod가 같은 row의 비율 확인

In [13]:
#비이탈자들의 date_modif_prod - date_activ
lt_0 = clean[clean["churn"]==0]["date_modif_prod"] - clean[clean["churn"]==0]["date_activ"]
print(f"비이탈자 소요시간 0의 비율: {(lt_0== 0).sum() / lt_0.shape[0]:.4f}")

#이탈자들의 date_modif_prod - date_activ
lt_1 = clean[clean["churn"]==1]["date_modif_prod"] - clean[clean["churn"]==1]["date_activ"]
print(f"이탈자 소요시간 0의 비율: {(lt_1== 0).sum() / lt_1.shape[0]:.4f}")

비이탈자 소요시간 0의 비율: 0.5034
이탈자 소요시간 0의 비율: 0.4301


In [14]:
#이것을 고려하여 결측치를 채움
import numpy as np
lt_0_rat = 0.5034
lt_0_mean = 1432

lt_1_rat = 0.4301
lt_1_mean = 1266

#비이탈자에서 0의 비율만큼 0 할당 
mask = clean['date_modif_prod'].isna() & (clean['churn'] == 0)
na_idx = clean[mask].index
n_total = len(na_idx)
n_zero = int(n_total * lt_0_rat)
zero_idx = np.random.choice(na_idx, size=n_zero, replace=False)
clean.loc[zero_idx, 'date_modif_prod'] = 0

#비이탈자에서 나머지는 date_activ+평균으로 할당
mask = clean["date_modif_prod"].isna() & (clean["churn"] == 0)
clean.loc[mask, "date_modif_prod"] = clean.loc[mask, "date_activ"] + lt_0_mean


#이탈자에서 0의 비율만큼 0 할당 
mask = clean['date_modif_prod'].isna() & (clean['churn'] == 1)
na_idx = clean[mask].index
n_total = len(na_idx)
n_zero = int(n_total * lt_1_rat)
zero_idx = np.random.choice(na_idx, size=n_zero, replace=False)
clean.loc[zero_idx, 'date_modif_prod'] = 0

#비이탈자에서 나머지는 date_activ+평균으로 할당
mask = clean["date_modif_prod"].isna() & (clean["churn"] == 1)
clean.loc[mask, "date_modif_prod"] = clean.loc[mask, "date_activ"] + lt_1_mean

In [15]:
# 데이터 조건
#date_activ, date_renewal <= date_end
#date_renewal은 특별한 특성이 없으니 데이터 조건을 만족하게끔 결측치 완성
#dr = (1- (dr-de / de-da)mean) *de

In [16]:
mean = ((clean["date_renewal"] - clean["date_end"]) / (clean["date_end"] - clean["date_activ"])).mean()
mask = clean["date_renewal"].isna()
clean.loc[mask, "date_renewal"] = ((1 - mean) * clean.loc[mask, "date_end"]).astype(int)

In [17]:
cols_to_fill = [
    'forecast_discount_energy', 'forecast_price_energy_p1',
    'forecast_price_energy_p2', 'forecast_price_pow_p1',
    'margin_gross_pow_ele', 'margin_net_pow_ele', 'net_margin',
    'origin_up', 'pow_max'
]

for col in cols_to_fill:
    if col in clean.columns:
        mode_val = clean[col].mode(dropna=True)
        if not mode_val.empty:
            clean[col].fillna(mode_val[0], inplace=True)

In [18]:
train_hist.head(3)
#var은 변동가, fix는 고정가

,id,price_date,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0


In [19]:
rows_with_null = train_hist[train_hist.isnull().any(axis=1)]
rows_with_null.head()

,id,price_date,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix
75,ef716222bbd97a8bdfcbb831e3575560,2015-04-01,NaN,NaN,NaN,NaN,NaN,NaN
221,0f5231100b2febab862f8dd8eaab3f43,2015-06-01,NaN,NaN,NaN,NaN,NaN,NaN
377,2f93639de582fadfbe3e86ce1c8d8f35,2015-06-01,NaN,NaN,NaN,NaN,NaN,NaN
413,f83c1ab1ca1d1802bb1df4d72820243c,2015-06-01,NaN,NaN,NaN,NaN,NaN,NaN
461,3076c6d4a060e12a049d1700d9b09cf3,2015-06-01,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
cols = [
    'price_p1_var', 'price_p2_var', 'price_p3_var',
    'price_p1_fix', 'price_p2_fix', 'price_p3_fix'
]
for col in cols:
    if col in train_hist.columns:
        mean_val = train_hist[col].mean()
        train_hist[col].fillna(mean_val, inplace=True)

In [21]:
train_hist['p1']=train_hist['price_p1_var']+train_hist['price_p1_fix']
train_hist['p2']=train_hist['price_p2_var']+train_hist['price_p2_fix']
train_hist['p3']=train_hist['price_p3_var']+train_hist['price_p3_fix']
train_hist.drop(['price_date','price_p1_var','price_p2_var','price_p3_var',
                'price_p1_fix','price_p2_fix','price_p3_fix'],inplace=True,axis=1)

In [22]:
clean = pd.merge(clean, train_hist, on="id")
clean = clean.drop(columns=["id"])

In [23]:
clean.shape

(193002, 29)

In [25]:
clean.to_csv("../dataSet/final_dataset.csv", index=False)